In [ ]:
from ultralytics import YOLO

In [ ]:
import torch

# Verifica si CUDA (GPU) está disponible
if torch.cuda.is_available():
    print("CUDA está disponible. GPU detectada:", torch.cuda.get_device_name(0))
else:
    print("CUDA no está disponible. Se usará la CPU.")


In [ ]:
torch.__version__

# Dataset 1,2 y 8

In [ ]:
import os
import shutil
import yaml
import cv2
import numpy as np
import random
from PIL import Image

## Configurando la copia de archivos

In [ ]:
# CONFIGURACIÓN DE RUTAS, DATASETS Y CLASES COMBINADAS

base_path = r"C:/Users/ALEX/OneDrive/Cursos/Maestría en Big Data y Data Science/Cursos-VIU/Oblgatorios/TFM/Datasets"

# Definición de cada dataset con su ruta, lista de nombres y mapeo a la lista combinada.
# Nota: En D8 se asume que "Miner" y "Rust" deben fusionarse con "miner" y "rust" de D1.
datasets = {
    "D1": {
        "path": os.path.join(base_path, "D1 - Coffee leaf diseases classification.v5i.yolov11"),
        "names": ['cerscospora', 'healthy', 'miner', 'phoma', 'rust'],
        "mapping": {0: 0, 1: 1, 2: 2, 3: 3, 4: 4}  # idéntico a la lista combinada
    },
    "D2": {
        "path": os.path.join(base_path, "D2- Hojas de cafe enfermedades.v2i.yolov11"),
        "names": ['Falta-de-boro', 'Falta-de-calcio', 'Falta-de-fosforo', 'Falta-de-hierro', 
                  'Falta-de-magnesio', 'Falta-de-manganeso', 'Falta-de-potasio', 'Falta-nitrogeno'],
        # Se mapea para que los índices de D2 sigan a los de D1 (0 a 4)
        "mapping": {0: 5, 1: 6, 2: 7, 3: 8, 4: 9, 5: 10, 6: 11, 7: 12}
    },
    "D8": {
        "path": os.path.join(base_path, "D8 - Coffee Leaves Detection.v1i.yolov11"),
        "names": ['Miner', 'Rust'],
        # Se mapea: "Miner" se asume igual que "miner" (índice 2) y "Rust" a "rust" (índice 4)
        "mapping": {0: 2, 1: 4}
    }
}

# Lista combinada de nombres (la unión de las clases, en el orden deseado)
combined_names = ['cerscospora', 'healthy', 'miner', 'phoma', 'rust',
                  'Falta-de-boro', 'Falta-de-calcio', 'Falta-de-fosforo', 'Falta-de-hierro', 
                  'Falta-de-magnesio', 'Falta-de-manganeso', 'Falta-de-potasio', 'Falta-nitrogeno']

In [ ]:
# CREAR ESTRUCTURA DE CARPETAS COMBINADA

# Se creará una única carpeta "combinado" con subcarpetas "train", "valid" y "test"
combined_path = os.path.join(base_path, "combinado")
for split in ["train", "valid", "test"]:
    os.makedirs(os.path.join(combined_path, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(combined_path, split, "labels"), exist_ok=True)

In [ ]:
combined_path

In [ ]:
combined_path = os.path.normpath(combined_path)

In [ ]:
# COPIAR Y REMAPEAR LOS ARCHIVOS DE CADA DATASET

def copy_and_remap_dataset(dataset_name, dataset_info):
    """
    Copia imágenes y etiquetas del dataset original a la estructura combinada.
    Renombra los archivos para evitar conflictos y remapea los índices de clase.
    """
    src_path = dataset_info["path"]
    mapping = dataset_info["mapping"]
    
    for split in ["train", "valid", "test"]:
        src_images = os.path.join(src_path, split, "images")
        src_labels = os.path.join(src_path, split, "labels")
        dst_images = os.path.join(combined_path, split, "images")
        dst_labels = os.path.join(combined_path, split, "labels")
        
        # Copiar imágenes (se renombra con el prefijo del dataset)
        if os.path.exists(src_images):
            for fname in os.listdir(src_images):
                src_file = os.path.join(src_images, fname)
                new_fname = f"{dataset_name}_{fname}"
                dst_file = os.path.join(dst_images, new_fname)
                shutil.copy2(src_file, dst_file)
        else:
            print(f"Advertencia: No existe la carpeta {src_images}")
        
        # Copiar etiquetas y remapear índices
        if os.path.exists(src_labels):
            for fname in os.listdir(src_labels):
                src_file = os.path.join(src_labels, fname)
                new_fname = f"{dataset_name}_{fname}"
                dst_file = os.path.join(dst_labels, new_fname)
                with open(src_file, 'r') as f:
                    lines = f.readlines()
                new_lines = []
                for line in lines:
                    parts = line.strip().split()
                    if parts:
                        try:
                            cls_old = int(parts[0])
                            cls_new = mapping.get(cls_old, cls_old)
                            new_line = str(cls_new) + " " + " ".join(parts[1:]) + "\n"
                            new_lines.append(new_line)
                        except Exception as e:
                            new_lines.append(line)
                    else:
                        new_lines.append(line)
                with open(dst_file, 'w') as f:
                    f.writelines(new_lines)
        else:
            print(f"Advertencia: No existe la carpeta {src_labels}")

# Procesar cada dataset
for ds in datasets:
    print(f"Procesando dataset {ds} ...")
    copy_and_remap_dataset(ds, datasets[ds])


In [ ]:
# CONTAR IMÁGENES POR CLASE EN TRAIN (CONJUNTO COMBINADO)

def count_images_per_class_combined(combined_path, combined_names):
    counts = {i: 0 for i in range(len(combined_names))}
    labels_dir = os.path.join(combined_path, "train", "labels")
    for fname in os.listdir(labels_dir):
        file_path = os.path.join(labels_dir, fname)
        try:
            with open(file_path, 'r') as f:
                lines = f.readlines()
            file_classes = set()
            for line in lines:
                parts = line.strip().split()
                if parts:
                    try:
                        cls = int(parts[0])
                        file_classes.add(cls)
                    except:
                        continue
            for cls in file_classes:
                if cls in counts:
                    counts[cls] += 1
        except Exception as e:
            print(f"Error leyendo {file_path}: {e}")
    return counts

initial_counts = count_images_per_class_combined(combined_path, combined_names)
print("Conteo inicial de imágenes por clase en TRAIN (combinado):")
for cls, count in initial_counts.items():
    print(f"  Clase {cls} ({combined_names[cls]}): {count} imágenes")

## Realizando el dataaumentation

In [ ]:
# DATA AUGMENTATION PARA HOMOGENIZAR LAS CLASES EN TRAIN

def find_image_file(image_dir, base_name):
    """
    Busca el archivo de imagen en image_dir que coincida con base_name
    comprobando las extensiones más comunes.
    """
    for ext in ['.jpg', '.jpeg', '.png']:
        candidate = os.path.join(image_dir, base_name + ext)
        if os.path.exists(candidate):
            return candidate
    return None

def imread_with_pil(path):
    try:
        with Image.open(path) as im:
            return np.array(im.convert("RGB"))
    except Exception as e:
        print("Error al leer con PIL:", path, e)
        return None

def augment_image_label(image_path, label_path, out_image_path, out_label_path, angle, brightness_factor):
    # Intentar leer con cv2
    img = cv2.imread(image_path)
    if img is None:
        # Si falla, intentar con PIL
        img = imread_with_pil(image_path)
        if img is None:
            print("Error al leer la imagen:", image_path)
            return
    
    h, w = img.shape[:2]
    
    # Aplicar rotación según el ángulo especificado
    if angle != 0:
        if angle == 90:
            img_aug = cv2.rotate(img, cv2.ROTATE_90_CLOCKWISE)
        elif angle == 180:
            img_aug = cv2.rotate(img, cv2.ROTATE_180)
        elif angle == 270:
            img_aug = cv2.rotate(img, cv2.ROTATE_90_COUNTERCLOCKWISE)
        else:
            img_aug = img.copy()
    else:
        img_aug = img.copy()
    
    # Ajustar brillo
    img_aug = np.clip(img_aug * brightness_factor, 0, 255).astype(np.uint8)
    
    # Guardar imagen aumentada
    cv2.imwrite(out_image_path, img_aug)
    
    # Procesar etiqueta: ajustar las coordenadas según la rotación
    new_lines = []
    if os.path.exists(label_path):
        with open(label_path, 'r') as f:
            lines = f.readlines()
        for line in lines:
            parts = line.strip().split()
            if parts and len(parts) == 5:
                try:
                    cls = parts[0]
                    x = float(parts[1])
                    y = float(parts[2])
                    bw = float(parts[3])
                    bh = float(parts[4])
                    # Ajustar coordenadas para rotaciones en formato YOLO
                    if angle == 90:
                        new_x = y
                        new_y = 1 - x
                        new_bw = bh
                        new_bh = bw
                    elif angle == 180:
                        new_x = 1 - x
                        new_y = 1 - y
                        new_bw = bw
                        new_bh = bh
                    elif angle == 270:
                        new_x = 1 - y
                        new_y = x
                        new_bw = bh
                        new_bh = bw
                    else:
                        new_x, new_y, new_bw, new_bh = x, y, bw, bh
                    new_line = f"{cls} {new_x:.6f} {new_y:.6f} {new_bw:.6f} {new_bh:.6f}\n"
                    new_lines.append(new_line)
                except Exception as e:
                    new_lines.append(line)
            else:
                new_lines.append(line)
        with open(out_label_path, 'w') as f:
            f.writelines(new_lines)
    else:
        open(out_label_path, 'w').close()

# Definir el objetivo: igualar al máximo número de imágenes por clase en TRAIN
current_counts = count_images_per_class_combined(combined_path, combined_names)
target_count = max(current_counts.values())
print(f"\nObjetivo de imágenes por clase (TRAIN): {target_count}")

train_images_dir = os.path.join(combined_path, "train", "images")
train_labels_dir = os.path.join(combined_path, "train", "labels")
label_files = os.listdir(train_labels_dir)

augmented_counter = 0
augmentation_round = 0
max_rounds = 8  # para evitar ciclos infinitos

while True:
    current_counts = count_images_per_class_combined(combined_path, combined_names)
    min_count = min(current_counts.values())
    print(f"Ronda de augmentación {augmentation_round}: mínimo actual = {min_count}")
    if min_count >= target_count or augmentation_round >= max_rounds:
        break
    # Recorre cada archivo de etiqueta en TRAIN
    for fname in label_files:
        label_path = os.path.join(train_labels_dir, fname)
        # Leer clases presentes en la imagen
        with open(label_path, 'r') as f:
            lines = f.readlines()
        if not lines:
            continue
        image_classes = set()
        for line in lines:
            parts = line.strip().split()
            if parts:
                try:
                    cls = int(parts[0])
                    image_classes.add(cls)
                except:
                    continue
        # Aquí se modifica la condición:
        # Solo se aumenta si PARA TODAS las clases de la imagen el conteo actual es menor que target_count.
        underrepresented = all(current_counts[cls] < target_count for cls in image_classes)
        if underrepresented:
            # Se supone que el nombre de la etiqueta y de la imagen son iguales (sin la extensión)
            base_name = os.path.splitext(fname)[0]
            # Buscar la imagen con extensión correcta
            image_path = find_image_file(train_images_dir, base_name)
            if image_path is None:
                print("No se encontró la imagen para:", base_name)
                continue
            # Nombres para los archivos aumentados
            aug_image_name = base_name + f"_aug{augmented_counter}"
            aug_label_name = base_name + f"_aug{augmented_counter}.txt"
            # Mantener la misma extensión que la imagen encontrada
            _, ext = os.path.splitext(image_path)
            out_image_path = os.path.join(train_images_dir, aug_image_name + ext)
            out_label_path = os.path.join(train_labels_dir, aug_label_name)
            # Parámetros aleatorios de augmentación
            angle = random.choice([0, 90, 180, 270])
            brightness_factor = random.uniform(0.7, 1.3)
            augment_image_label(image_path, label_path, out_image_path, out_label_path, angle, brightness_factor)
            augmented_counter += 1
    augmentation_round += 1

final_counts = count_images_per_class_combined(combined_path, combined_names)
print("\nConteo final de imágenes por clase en TRAIN (combinado):")
for cls, count in final_counts.items():
    print(f"  Clase {cls} ({combined_names[cls]}): {count} imágenes")

In [ ]:
initial_counts = count_images_per_class_combined(combined_path, combined_names)
print("Conteo final de imágenes por clase en TRAIN (combinado):")
for cls, count in initial_counts.items():
    print(f"  Clase {cls} ({combined_names[cls]}): {count} imágenes")

In [ ]:
# GENERAR data.yaml COMBINADO

data_yaml = {
    "train": os.path.join(combined_path, "train", "images").replace("\\", "/"),
    "val": os.path.join(combined_path, "valid", "images").replace("\\", "/"),
    "test": os.path.join(combined_path, "test", "images").replace("\\", "/"),
    "nc": len(combined_names),
    "names": combined_names
}
yaml_path = os.path.join(combined_path, "data.yaml")
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, sort_keys=False)
print("\ndata.yaml combinado generado en:", yaml_path)

## Realizar entrenamiento

In [ ]:
import os
import time
import random
import numpy as np
import pandas as pd
import yaml
from ultralytics import YOLO
import onnxruntime as ort
import shutil
import subprocess
from ptflops import get_model_complexity_info

### Funciones a utilizar

In [ ]:
# Evaluación y extracción de métricas
def extract_metrics(model):
    """
    Evalúa el modelo usando model.val() y extrae las métricas de validación.
    
    Si el objeto retornado no es un diccionario, se intenta extraer atributos públicos.
    Si no se obtiene una métrica clave (por ejemplo, "metrics/precision(B)"), se intenta leer
    la última fila del CSV de resultados ubicado en 'runs/detect/train/results.csv'.
    
    Retorna:
      - metrics_dict (dict): Diccionario con las métricas.
    """
    metrics_obj = model.val()
    if isinstance(metrics_obj, dict):
        metrics_dict = metrics_obj
    else:
        try:
            metrics_dict = metrics_obj.results_dict()
        except Exception as e:
            print("No se pudo usar results_dict(), se intentará extrayendo atributos.", e)
            try:
                metrics_dict = {attr: getattr(metrics_obj, attr) 
                                for attr in dir(metrics_obj)
                                if not attr.startswith('_') and not callable(getattr(metrics_obj, attr))}
            except Exception as e:
                print("Error extrayendo atributos del objeto de validación:", e)
                metrics_dict = {}
    
    # Verificar si se obtuvo la métrica clave; de lo contrario, leer el CSV
    if not metrics_dict or "metrics/precision(B)" not in metrics_dict:
        try:
            csv_path = r"runs\detect\train\results.csv"
            if os.path.exists(csv_path):
                results_df = pd.read_csv(csv_path)
                # Se toma la última fila, que corresponde a las métricas finales
                final_metrics = results_df.iloc[-1].to_dict()
                metrics_dict = final_metrics
        except Exception as e:
            print("Error leyendo el CSV de resultados:", e)
    return metrics_dict

In [ ]:
# Exportar el modelo a ONNX y copiarlo con el nombre deseado
def export_and_copy_onnx(model, model_name, imgsz, opset=12):
    """
    Exporta el modelo a ONNX usando model.export(). Por defecto, Ultralytics guarda el ONNX
    como 'best.onnx' en 'runs/detect/train/weights'. Esta función espera a que dicho archivo exista,
    lo copia a la carpeta 'exports' con el nombre f"{model_name}.onnx" y retorna la ruta de destino.
    
    Se fuerza el parámetro opset para evitar problemas de compatibilidad (por ejemplo, Constant version 19).
    
    Parámetros:
        - model: Instancia del modelo YOLO.
        - model_name (str): Nombre identificativo del modelo.
        - imgsz (int): Tamaño de la imagen.
        - opset (int): Versión del opset para exportar (por ejemplo, 12).
    
    Retorna:
        - onnx_dest_path (str): Ruta al archivo ONNX copiado en la carpeta 'exports'.
    """
    onnx_dest_path = os.path.join("exports", f"{model_name}.onnx")
    os.makedirs("exports", exist_ok=True)
    
    # Exporta el modelo con el opset especificado
    model.export(format="onnx", imgsz=imgsz, exist_ok=True, opset=opset)
    
    default_onnx_path = os.path.join("runs", "detect", "train", "weights", "best.onnx")
    timeout = 60
    start_time = time.time()
    # Esperar a que default_onnx_path se cree
    while not os.path.exists(default_onnx_path) and (time.time() - start_time) < timeout:
        time.sleep(1)
    if os.path.exists(default_onnx_path):
        shutil.copy2(default_onnx_path, onnx_dest_path)
    else:
        print(f"Error: No se encontró el archivo ONNX por defecto en {default_onnx_path}")
    
    # Esperar a que el archivo copiado exista en exports
    start_time = time.time()
    while not os.path.exists(onnx_dest_path) and (time.time() - start_time) < timeout:
        time.sleep(1)
    if not os.path.exists(onnx_dest_path):
        print(f"Error: No se encontró el archivo ONNX en {onnx_dest_path}")
    
    return onnx_dest_path

In [ ]:
# Exportar el modelo a TensorRT y copiarlo con el nombre deseado
def export_and_copy_trt(model, model_name, imgsz, opset=12):
    """
    Exporta el modelo a TensorRT usando model.export(format="trt"). Por defecto, 
    Ultralytics guarda el engine en 'runs/detect/train/weights' como 'best.trt'. 
    Esta función espera a que dicho archivo exista, lo copia a la carpeta 'exports' con el nombre
    f"{model_name}.trt" y retorna la ruta de destino.
    
    Se fuerza el parámetro opset para la conversión. Si tensorrt no está instalado, se captura la excepción.
    
    Parámetros:
        - model: Instancia del modelo YOLO.
        - model_name (str): Nombre identificativo del modelo.
        - imgsz (int): Tamaño de la imagen.
        - opset (int): Versión del opset para exportar.
    
    Retorna:
        - trt_dest_path (str) o None.
    """
    try:
        model.export(format="trt", imgsz=imgsz, exist_ok=True, opset=opset)
    except ModuleNotFoundError as e:
        print("TensorRT no está instalado. Se retornará None para la velocidad TRT.")
        return None
    except Exception as e:
        print("Error exportando a TensorRT:", e)
        return None

    trt_dest_path = os.path.join("exports", f"{model_name}.trt")
    os.makedirs("exports", exist_ok=True)
    
    default_trt_path = os.path.join("runs", "detect", "train", "weights", "best.trt")
    timeout = 60
    start_time = time.time()
    while not os.path.exists(default_trt_path) and (time.time() - start_time) < timeout:
        time.sleep(1)
    if os.path.exists(default_trt_path):
        shutil.copy2(default_trt_path, trt_dest_path)
    else:
        print(f"Error: No se encontró el archivo TRT por defecto en {default_trt_path}")
    
    start_time = time.time()
    while not os.path.exists(trt_dest_path) and (time.time() - start_time) < timeout:
        time.sleep(1)
    if not os.path.exists(trt_dest_path):
        print(f"Error: No se encontró el archivo TRT en {trt_dest_path}")
        return None
    return trt_dest_path

In [ ]:
# Función para exportar y copiar el checkpoint en formato PT
def export_and_copy_pt(model, model_name):
    """
    Copia el checkpoint del modelo en formato .pt. Por defecto, Ultralytics guarda el checkpoint
    como 'best.pt' en 'runs/detect/train/weights'. Esta función espera a que dicho archivo exista,
    lo copia a la carpeta 'exports' con el nombre f"{model_name}.pt" y retorna la ruta de destino.
    
    Retorna:
      - pt_dest_path (str): Ruta al archivo .pt copiado en la carpeta 'exports'.
    """
    pt_dest_path = os.path.join("exports", f"{model_name}.pt")
    os.makedirs("exports", exist_ok=True)
    
    default_pt_path = os.path.join("runs", "detect", "train", "weights", "best.pt")
    timeout = 60
    start_time = time.time()
    # Esperar a que se cree el checkpoint por defecto
    while not os.path.exists(default_pt_path) and (time.time() - start_time) < timeout:
        time.sleep(1)
    if os.path.exists(default_pt_path):
        shutil.copy2(default_pt_path, pt_dest_path)
    else:
        print(f"Error: No se encontró el archivo PT por defecto en {default_pt_path}")
    
    # Esperar a que el archivo copiado exista en exports
    start_time = time.time()
    while not os.path.exists(pt_dest_path) and (time.time() - start_time) < timeout:
        time.sleep(1)
    if not os.path.exists(pt_dest_path):
        print(f"Error: No se encontró el archivo PT en {pt_dest_path}")
    
    return pt_dest_path

In [ ]:
# Función para medir la velocidad de inferencia en CPU usando ONNX
def measure_onnx_inference_speed(onnx_path, imgsz, n_iter=20):
    """
    Mide la velocidad de inferencia en CPU usando onnxruntime.
    
    Parámetros:
      - onnx_path (str): Ruta al archivo ONNX.
      - imgsz (int): Tamaño de la imagen (ancho y alto).
      - n_iter (int): Número de iteraciones para promediar.
    
    Retorna:
      - Tiempo promedio en milisegundos.
    """
    try:
        ort_session = ort.InferenceSession(onnx_path)
        dummy_input = np.random.rand(1, 3, imgsz, imgsz).astype(np.float32)
        # Warm-up
        for _ in range(5):
            _ = ort_session.run(None, {"images": dummy_input})
        start = time.time()
        for _ in range(n_iter):
            _ = ort_session.run(None, {"images": dummy_input})
        end = time.time()
        avg_time = (end - start) / n_iter * 1000  # ms
        return avg_time
    except Exception as e:
        print(f"Error midiendo ONNX runtime: {e}")
        return None

In [ ]:
# Medir la velocidad de inferencia en T4 TensorRT usando trtexec
def measure_trt_inference_speed(trt_engine_path, n_iter=20):
    """
    Mide la velocidad de inferencia en una GPU T4 utilizando la herramienta 'trtexec'.
    Se asume que 'trtexec' está instalado y en el PATH.
    
    Parámetros:
      - trt_engine_path (str): Ruta al engine de TensorRT (archivo .trt).
      - n_iter (int): Número de iteraciones para promediar.
    
    Retorna:
      - Tiempo promedio en milisegundos (float) o None en caso de error.
    """
    if trt_engine_path is None:
        return None
    try:
        cmd = ["trtexec", f"--loadEngine={trt_engine_path}", f"--iterations={n_iter}"]
        proc = subprocess.Popen(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, universal_newlines=True)
        stdout, stderr = proc.communicate(timeout=120)
        avg_time = None
        for line in stdout.splitlines():
            if "Average latency" in line:
                parts = line.split(":")
                if len(parts) > 1:
                    avg_time = float(parts[1].strip().split()[0])
                    break
        if avg_time is None:
            print("No se pudo extraer el tiempo promedio de la salida de trtexec.")
        return avg_time
    except Exception as e:
        print(f"Error midiendo TensorRT runtime: {e}")
        return None

In [ ]:
# Calular los flops
def compute_flops(model, imgsz):
    # get_model_complexity_info devuelve un string y el número total de FLOPs
    # Asegúrate de que model.model es el objeto PyTorch con el que se pueda contar FLOPs.
    macs, params = get_model_complexity_info(model.model, (3, imgsz, imgsz), as_strings=False, print_per_layer_stat=False)
    flops = 2 * macs  # MACs a FLOPs (multiplicamos por 2)
    return flops

In [ ]:
# Calcular parámetros y combinar todos los resultados
def compute_model_params(model):
    """
    Calcula la cantidad de parámetros del modelo en millones.
    
    Retorna:
      - params (float): Número de parámetros en millones.
    """
    return sum(p.numel() for p in model.model.parameters()) / 1e6

def combine_results(model_name, imgsz, metrics_dict, avg_time_cpu, avg_time_trt, params, flops):
    """
    Combina el nombre del modelo, el tamaño de la imagen, las métricas, la velocidad de inferencia,
    la cantidad de parámetros y los FLOPs en un único diccionario.
    
    Retorna:
      - results (dict): Diccionario con toda la información.
    """
    results = {
        "Modelo": model_name,
        "Tamaño (píxeles)": imgsz,
    }
    results.update(metrics_dict)
    results.update({
        "Velocidad CPU ONNX (ms)": avg_time_cpu,
        "Velocidad T4 TensorRT (ms)": avg_time_trt,
        "Parámetros (M)": params,
        "FLOPs (B)": flops
    })
    return results

### Configuraciones iniciales

In [ ]:
# Configuración de Fine Tuning y evaluación de modelos

# Lista de modelos a fine-tunear. Debes tener disponibles los pesos preentrenados.
models_info = [
    {"name": "YOLO11n", "weights": "yolo11n.pt"},
    {"name": "YOLO12n", "weights": "yolo12n.pt"},
    {"name": "YOLO11s", "weights": "yolo11s.pt"},
    {"name": "YOLO12s", "weights": "yolo12s.pt"}
]

# Parámetros de entrenamiento
epochs = 50
imgsz = 640
batch = 16
patience = 5
device = 'cuda:0'

# Ruta al archivo data.yaml del dataset combinado
yaml_path = 'C:/Users/ALEX/OneDrive/Cursos/Maestría en Big Data y Data Science/Cursos-VIU/Oblgatorios/TFM/detection-diseases-coffee/combinado/data.yaml'

results_list = []

### Entrenamiento de YOLO11n

In [ ]:
# Procesar el primer modelo: YOLO11n
info = models_info[0]
model_name = info["name"]
weights_path = info["weights"]
model = YOLO(weights_path)

# 1. Entrenamiento del modelo (fine tuning)
train_results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        patience=patience,
        device=device,
        exist_ok=True,
        pretrained=True
        )

In [ ]:
# 2. Evaluación y extracción de métricas
metrics_dict = extract_metrics(model)

metrics_dict

In [ ]:
# Guardar ruta de exortación en formato pt
pt_export_path = export_and_copy_pt(model, model_name)

# 3. Exportar a ONNX y copiarlo con el nombre deseado
onnx_export_path = export_and_copy_onnx(model, model_name, imgsz)
avg_time_cpu = measure_onnx_inference_speed(onnx_export_path, imgsz)

# 4. Exportar a TensorRT y medir velocidad (si es posible)
trt_engine_path = export_and_copy_trt(model, model_name, imgsz)
avg_time_trt = measure_trt_inference_speed(trt_engine_path, n_iter=20)

# 5. Calcular parámetros y definir placeholder para FLOPs
params = compute_model_params(model)

flops = compute_flops(model, imgsz)

# 6. Combinar toda la información
results_yolo11n = combine_results(model_name, imgsz, metrics_dict, avg_time_cpu, avg_time_trt, params, flops)

results_list.append(results_yolo11n)

results_list

### Entrenamiento de YOLO12n

In [ ]:
# Procesar el primer modelo: YOLO12n
info = models_info[1]
model_name = info["name"]
weights_path = info["weights"]
model = YOLO(weights_path)

# 1. Entrenamiento del modelo:
train_results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        patience=patience,
        device=device,
        exist_ok=True,
        pretrained=True)

In [ ]:
# 2. Evaluación y extracción de métricas
metrics_dict = extract_metrics(model)

metrics_dict

In [ ]:
# Guardar ruta de exortación en formato pt
pt_export_path = export_and_copy_pt(model, model_name)

# 3. Exportar a ONNX y copiarlo con el nombre deseado
onnx_export_path = export_and_copy_onnx(model, model_name, imgsz)
avg_time_cpu = measure_onnx_inference_speed(onnx_export_path, imgsz)

# 4. Exportar a TensorRT y medir velocidad (si es posible)
trt_engine_path = export_and_copy_trt(model, model_name, imgsz)
avg_time_trt = measure_trt_inference_speed(trt_engine_path, n_iter=20)

# 5. Calcular parámetros y definir placeholder para FLOPs
params = compute_model_params(model)

flops = compute_flops(model, imgsz)

# 6. Combinar toda la información
results_yolo12n = combine_results(model_name, imgsz, metrics_dict, avg_time_cpu, avg_time_trt, params, flops)

results_list.append(results_yolo12n)

results_list

### Entrenamiento de YOLO11s

In [ ]:
# Procesar el primer modelo: YOLO11s
info = models_info[2]
model_name = info["name"]
weights_path = info["weights"]
model = YOLO(weights_path)

# 1. Entrenamiento del modelo:
train_results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        patience=patience,
        device=device,
        exist_ok=True,
        pretrained=True)

In [ ]:
# 2. Evaluación y extracción de métricas
metrics_dict = extract_metrics(model)

metrics_dict

In [ ]:
# Guardar ruta de exortación en formato pt
pt_export_path = export_and_copy_pt(model, model_name)

# 3. Exportar a ONNX y copiarlo con el nombre deseado
onnx_export_path = export_and_copy_onnx(model, model_name, imgsz)
avg_time_cpu = measure_onnx_inference_speed(onnx_export_path, imgsz)

# 4. Exportar a TensorRT y medir velocidad (si es posible)
trt_engine_path = export_and_copy_trt(model, model_name, imgsz)
avg_time_trt = measure_trt_inference_speed(trt_engine_path, n_iter=20)

# 5. Calcular parámetros y definir placeholder para FLOPs
params = compute_model_params(model)

flops = compute_flops(model, imgsz)

# 6. Combinar toda la información
results_yolo11s = combine_results(model_name, imgsz, metrics_dict, avg_time_cpu, avg_time_trt, params, flops)

results_list.append(results_yolo11s)

results_list

### Entrenamiento de YOLO12s

In [ ]:
# Procesar el primer modelo: YOLO12s
info = models_info[3]
model_name = info["name"]
weights_path = info["weights"]
model = YOLO(weights_path)

# 1. Entrenamiento del modelo:
train_results = model.train(
        data=yaml_path,
        epochs=epochs,
        imgsz=imgsz,
        batch=batch,
        patience=patience,
        device=device,
        exist_ok=True,
        pretrained=True)

In [ ]:
# 2. Evaluación y extracción de métricas
metrics_dict = extract_metrics(model)

metrics_dict

In [ ]:
# Guardar ruta de exortación en formato pt
pt_export_path = export_and_copy_pt(model, model_name)

# 3. Exportar a ONNX y copiarlo con el nombre deseado
onnx_export_path = export_and_copy_onnx(model, model_name, imgsz)
avg_time_cpu = measure_onnx_inference_speed(onnx_export_path, imgsz)

# 4. Exportar a TensorRT y medir velocidad (si es posible)
trt_engine_path = export_and_copy_trt(model, model_name, imgsz)
avg_time_trt = measure_trt_inference_speed(trt_engine_path, n_iter=20)

# 5. Calcular parámetros y definir placeholder para FLOPs
params = compute_model_params(model)

flops = compute_flops(model, imgsz)

# 6. Combinar toda la información
results_yolo12s = combine_results(model_name, imgsz, metrics_dict, avg_time_cpu, avg_time_trt, params, flops)

results_list.append(results_yolo12s)

results_list

### Revisión de resultados

In [ ]:
# Crear y mostrar la tabla comparativa
df = pd.DataFrame(results_list)

print("\nComparación de modelos fine-tuned:")
df

In [ ]:
# Guardar la tabla en CSV
df.to_csv("comparacion_modelos.csv", index=False)

In [ ]:
def select_relevant_metrics(df,keywords):
    """
    A partir de un DataFrame con columnas de métricas (por ejemplo, provenientes del CSV de resultados)
    detecta cuáles son las métricas más relevantes para comparar modelos de detección y genera un texto
    explicativo de la elección.
    
    Se consideran relevantes las columnas que contengan las palabras clave:
      - precision
      - recall
      - mAP50-95
      - mAP50
    Adicionalmente, se sugieren opcionalmente columnas como 'epoch' y 'time' para analizar eficiencia.

    Retorna:
      - relevant_metrics: lista de nombres de columnas relevantes
      - optional_metrics: lista de columnas opcionales (como 'epoch' o 'time')
      - explanation: texto explicativo de la elección
    """
    # Palabras clave para métricas principales (sin importar mayúsculas o minúsculas)
    relevant_metrics = []
    for col in df.columns:
        for key in keywords:
            if key.lower() in col.lower():
                relevant_metrics.append(col)
                break

    # Métricas opcionales que se pueden usar para comparar eficiencia o duración
    optional_metrics = []
    for opt in ['time', 'epoch']:
        if opt in df.columns:
            optional_metrics.append(opt)
    
    explanation = "Se han seleccionado las siguientes métricas principales para comparar el desempeño de detección:\n"
    explanation += ", ".join(relevant_metrics) + ".\n\n"
    explanation += "Estas métricas se consideran relevantes porque:\n"
    explanation += "- Las métricas con el prefijo 'metrics/' (por ejemplo, " + ", ".join([m for m in relevant_metrics if 'metrics/' in m]) + ") " 
    explanation += "representan directamente la calidad de la detección (precisión, recall y mAP) en el conjunto de validación.\n"
    explanation += "- Los valores de 'mAP50' y 'mAP50-95' permiten evaluar el desempeño en diferentes umbrales de IoU, lo cual es crucial en tareas de detección.\n\n"
    explanation += "Adicionalmente, se consideran opcionales las siguientes columnas para analizar la eficiencia y convergencia:\n"
    explanation += ", ".join(optional_metrics) + ".\n"
    explanation += "Estas columnas indican el tiempo de entrenamiento y el número de epochs, lo que ayuda a comparar la velocidad de convergencia y el costo computacional, aunque no son indicadores directos de la calidad en inferencia.\n\n"
    explanation += "Por otro lado, se omiten columnas con prefijos como 'train/' (pérdidas de entrenamiento) y 'lr/' (tasa de aprendizaje) ya que, aunque son útiles para monitorizar el proceso de optimización, no reflejan directamente el desempeño final en la tarea de detección."
    
    return relevant_metrics, optional_metrics, explanation

In [ ]:
keywords = ['precision', 'recall', 'mAP50-95', 'mAP50']
relevant1, optional, text_explanation = select_relevant_metrics(df,keywords)
print(text_explanation)

In [ ]:
keywords = ['Velocidad CPU ONNX (ms)','Parámetros (M)','FLOPs (B)']
relevant2, optional, text_explanation = select_relevant_metrics(df,keywords)
print(text_explanation)

In [ ]:
df[["Modelo","epoch"] + relevant1 + relevant2]

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_grouped_bars_and_explain(df, relevant_metrics, title="Comparación de métricas entre modelos"):
    """
    Genera un gráfico de barras agrupadas para las métricas indicadas, comparando cada modelo.
    Además, produce un texto explicativo indicando cuál modelo es mejor en cada métrica.

    Parámetros:
      - df (DataFrame): Debe contener una columna "Modelo" y columnas de métricas numéricas.
      - relevant_metrics (list): Lista de nombres de columnas de métricas a graficar.
      - title (str): Título opcional para el gráfico.

    Retorna:
      - explanation (str): Un texto que explica cuál modelo es mejor en cada métrica.
    """
    # 1. Subconjunto con "Modelo" y las métricas relevantes
    subset = df[["Modelo"] + relevant_metrics].copy()

    # 2. Establecer "Modelo" como índice para facilitar el gráfico
    subset.set_index("Modelo", inplace=True)

    # 3. Crear gráfico de barras agrupadas
    ax = subset.plot(kind="bar", figsize=(10 + 1.5*len(relevant_metrics), 6), rot=0, title=title)
    ax.set_ylabel("Valor de la métrica")
    # Mover la leyenda para que no tape el gráfico
    plt.legend(title="Métricas", bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    plt.show()

    # 4. Generar explicación sobre los resultados
    explanation = f"En este gráfico se comparan las siguientes métricas: {', '.join(relevant_metrics)}.\n"
    explanation += "Cada grupo de barras corresponde a un modelo, y cada color representa una métrica distinta.\n\n"

    # 5. Determinar el/los mejor(es) modelo(s) para cada métrica
    best_models = {}
    for metric in relevant_metrics:
        # Buscar el valor máximo
        max_value = subset[metric].max()
        # Encontrar qué modelo(s) tienen ese valor máximo
        best_mods = subset[subset[metric] == max_value].index.tolist()
        best_models[metric] = (best_mods, max_value)

    # 6. Construir texto explicativo de cuál es mejor en cada métrica
    for metric, (best_mods, max_value) in best_models.items():
        if len(best_mods) == 1:
            explanation += f"- Para la métrica '{metric}', el mejor modelo es **{best_mods[0]}** con un valor de **{max_value:.4f}**.\n"
        else:
            explanation += f"- Para la métrica '{metric}', los mejores modelos son **{', '.join(best_mods)}** con un valor de **{max_value:.4f}**.\n"

    # 7. Resumen: cuántas veces cada modelo fue el mejor
    count_best = {}
    for metric, (best_mods, _) in best_models.items():
        for m in best_mods:
            count_best[m] = count_best.get(m, 0) + 1

    explanation += "\nResumen de cuántas veces cada modelo obtuvo el mejor valor:\n"
    for m, c in sorted(count_best.items(), key=lambda x: x[1], reverse=True):
        explanation += f"  - {m}: {c} métrica(s)\n"

    explanation += "\nEn base a estas métricas, el modelo con mayor cantidad de 'mejores resultados' se podría considerar superior. " \
                   "No obstante, la elección final puede depender de otras consideraciones (velocidad de inferencia, tamaño, etc.)."

    return explanation

explanation_text = plot_grouped_bars_and_explain(df, relevant1)
print(explanation_text)


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

def plot_model_comparison(df, relevant_metrics):
    """
    Genera un gráfico de barras para cada métrica relevante comparando los modelos.
    
    Parámetros:
      - df: DataFrame que contiene los resultados por modelo.
      - relevant_metrics: Lista de columnas (métricas) a graficar.
    """
    num_metrics = len(relevant_metrics)
    # Si hay varias métricas, creamos subplots en una fila.
    fig, axes = plt.subplots(nrows=1, ncols=num_metrics, figsize=(5*num_metrics, 5))
    
    # Si sólo hay una métrica, aseguramos que axes sea una lista.
    if num_metrics == 1:
        axes = [axes]
        
    for ax, metric in zip(axes, relevant_metrics):
        # Verifica que la columna sea numérica para graficarla
        if pd.api.types.is_numeric_dtype(df[metric]):
            ax.bar(df["Modelo"], df[metric], color='skyblue')
            ax.set_title(metric)
            ax.set_xlabel("Modelo")
            ax.set_ylabel(metric)
            ax.tick_params(axis='x', rotation=45)
        else:
            ax.text(0.5, 0.5, f"No es numérica: {metric}", horizontalalignment='center', verticalalignment='center')
            ax.set_axis_off()
    plt.tight_layout()
    plt.show()

plot_model_comparison(df, relevant1)

### Análisis de resultados

1. **Métricas principales (precisión, recall, mAP50 y mAP50-95)**  
   - **YOLO11s** alcanza la mayor precisión (\(0.8074\)), así como los mejores valores de mAP50 (\(0.8679\)) y mAP50-95 (\(0.8167\)). Esto indica que, en general, YOLO11s tiene la mayor capacidad para detectar correctamente los objetos (alta precisión) y un rendimiento sólido a diferentes umbrales de IoU (mAP).  
   - **YOLO11n**, por otro lado, obtiene el mejor recall (\(0.8399\)). Esto significa que YOLO11n tiende a detectar un mayor porcentaje de objetos, aunque su precisión sea ligeramente menor.  
   - **YOLO12n** y **YOLO12s** no superan a YOLO11s ni a YOLO11n en estas métricas específicas. YOLO12n tiene un recall un poco inferior a YOLO11n y una precisión ligeramente mayor que YOLO11n, pero no alcanza los valores de YOLO11s. Por su parte, YOLO12s queda algo rezagado en todas las métricas consideradas.

2. **Tamaño del modelo (número de parámetros)**  
   - **YOLO11s** y **YOLO12s** presentan aproximadamente 9.4M y 9.2M parámetros, respectivamente, mientras que **YOLO11n** y **YOLO12n** rondan los 2.5M. Esto implica que los modelos "s" (small) son significativamente más grandes que los "n" (nano) en términos de capacidad y, potencialmente, de coste computacional.  
   - El hecho de que YOLO11s sea el modelo con mejor rendimiento coincide con que sea también uno de los más grandes, lo que sugiere que la mayor capacidad de parámetros podría estar aprovechándose para obtener un mejor ajuste a los datos.

3. **Número de épocas (epoch)**  
   - YOLO11s entrenó durante 42 épocas, mientras que YOLO12s solo 24. Esto puede indicar que YOLO12s no alcanzó su punto óptimo de entrenamiento. Del mismo modo, YOLO11n y YOLO12n entrenaron 27 y 33 épocas, respectivamente.  
   - Podría ser interesante homogeneizar el número de épocas o aplicar técnicas de early stopping consistentes para comparar los modelos en igualdad de condiciones de entrenamiento.

4. **Conclusión general**  
   - **YOLO11s** es el modelo con mejor rendimiento global en las métricas de precisión, mAP50 y mAP50-95. Sin embargo, es también el más grande en número de parámetros (junto con YOLO12s), lo que implica mayor coste computacional en entrenamiento e inferencia.  
   - **YOLO11n** destaca por su alto recall, lo que podría ser valioso en aplicaciones donde es preferible detectar tantos objetos como sea posible (aunque a costa de más falsos positivos). Además, YOLO11n es mucho más ligero que YOLO11s.  
   - **YOLO12n** y **YOLO12s** no superan los resultados de sus contrapartes "11" en las métricas analizadas, si bien YOLO12n tiene un rendimiento cercano a YOLO11n y también un número de parámetros similar.  
   - A la hora de elegir un modelo, es importante balancear la precisión, el recall y el mAP con la complejidad (número de parámetros) y la velocidad de inferencia. En este caso, si la prioridad absoluta es la calidad de detección, YOLO11s es el mejor de los cuatro; si la prioridad es un modelo ligero con alto recall, YOLO11n podría ser la mejor elección.


### Predicción

In [ ]:
import torch
from ultralytics import YOLO
import os

# Cargar el modelo YOLO
model_path = "exports//YOLO11n.pt"
modelo_pytorch = YOLO(model_path)  # Carga la arquitectura + pesos

# Definir la carpeta de salida para predicciones
pred_dir = r"C:\Users\ALEX\OneDrive\Cursos\Maestría en Big Data y Data Science\Cursos-VIU\Oblgatorios\TFM\Datasets\predicciones\comb"
os.makedirs(pred_dir, exist_ok=True)  # Crear carpeta si no existe

# Realizar predicciones y guardarlas en la carpeta de destino
predictions = modelo_pytorch.predict(
    source=r"C:\Users\ALEX\OneDrive\Cursos\Maestría en Big Data y Data Science\Cursos-VIU\Oblgatorios\TFM\detection-diseases-coffee\imagenes de prueba",
    imgsz=640,
    save=True,         # Guarda automáticamente las imágenes con anotaciones
    project=pred_dir,  # Guarda los resultados en la carpeta personalizada
    name="predicciones"  # Nombre del subdirectorio dentro de `pred_dir`
)

print(f"Predicciones guardadas en: {pred_dir}/predicciones")

## Exportación en tflite

In [ ]:
import os
os.environ["PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION"] = "python"
import onnx
from onnx_tf.backend import prepare
import tensorflow as tf

print("TensorFlow version:", tf.__version__)

In [ ]:
# Ruta para el modelo ONNX
onnx_path = "C://Users//ALEX//OneDrive//Cursos//Maestría en Big Data y Data Science//Cursos-VIU//Oblgatorios//TFM//detection-diseases-coffee//exports//YOLO11n.onnx"

# Cargar el modelo ONNX
onnx_model = onnx.load(onnx_path)

# Preparar la representación de TensorFlow
tf_rep = prepare(onnx_model)

In [ ]:
# Obtener la información de los tensores de salida
output_tensor = onnx_model.graph.output[0]
output_shape = [dim.dim_value for dim in output_tensor.type.tensor_type.shape.dim]

# La última dimensión suele ser el número de clases
num_classes_onnx = output_shape[-1]

print(f"Cantidad de clases en el modelo ONNX: {num_classes_onnx}")

In [ ]:
saved_model_dir_base = "C://TEMP_MODEL_YOLO11n" #  Ruta MUY corta y simple
saved_model_dir = os.path.join(saved_model_dir_base, "OUT") # Subdirectorio también muy corto

# Asegurarse de que el directorio base exista 
os.makedirs(saved_model_dir_base, exist_ok=True) # Asegura que el directorio "Modelo" existe
# Asegurarse de que el directorio de salida del SavedModel exista
os.makedirs(saved_model_dir, exist_ok=True)

In [ ]:
# Exportar el SavedModel
tf_rep.export_graph(saved_model_dir)
print("Modelo convertido a TensorFlow SavedModel en:", saved_model_dir)

In [ ]:
import tensorflow as tf

# Cargar el modelo usando SavedModel API
model = tf.saved_model.load(r"C:\TEMP_MODEL_YOLO11n\OUT")

# Verificar las firmas del modelo
print("Firmas del modelo:", list(model.signatures.keys()))



In [ ]:
# Obtener la salida del modelo desde la firma "serving_default"
output_shape = list(model.signatures["serving_default"].structured_outputs.values())[0].shape
num_classes = output_shape[-1]  # Última dimensión representa las clases en clasificación

print(f"Cantidad de clases en el modelo SavedModel: {num_classes}")

In [ ]:
# Convertir el SavedModel a TFLite
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Guardar el modelo TFLite
tflite_path = r"Modelo\YOLO11n.tflite"
with open(tflite_path, "wb") as f:
    f.write(tflite_model)
print("Modelo exportado a TFLite y guardado en:", tflite_path)